Card Deck probability of things

In [1]:
import torch

suits = torch.tensor(range(4))
ranks = torch.tensor([(i + 2) for i in range(13)])

deck = torch.cartesian_prod(suits,ranks)

def probOfRank(deck, rank):
    foundRanks = torch.where(deck == rank)[1].nonzero()
    return foundRanks.size()[0] / deck.size()[0]

king = 13
print(probOfRank(deck, king))

0.07692307692307693


Permutation & combination function

In [2]:
import torch
import itertools

def permutations(perms, tensor):
    return torch.tensor(list(itertools.permutations(tensor.tolist(), perms)))

def combinations(perms, tensor):
    return torch.combinations(tensor, perms)

print(permutations(2, torch.tensor([(i + 1) for i in range(5)])).size()[0])
print(combinations(2, torch.tensor([(i + 1) for i in range(5)])).size()[0])

20
10


Probability: First and second card king with permutations

In [3]:
hands = permutations(2, deck)

king = 13
# all perms, keep only first hand, take the rank
firstHandRanks = hands[:, 0, 1]
firstHandKings = torch.where(firstHandRanks == king, torch.ones(1), torch.zeros(1)).nonzero()

probFirstHandKings = firstHandKings.size()[0] / hands.size()[0]
print(probFirstHandKings)

handsWithFirstHandIsKing = torch.index_select(hands, 0, firstHandKings[:,0])
handsWithTwoKings = torch.where(handsWithFirstHandIsKing[:, 1, 1] == king, torch.ones(1), torch.zeros(1)).nonzero()
probTwoKings = handsWithTwoKings.size()[0] / hands.size()[0]
print(probTwoKings)

probSecondKingAfterFirstKing = probTwoKings / probFirstHandKings

print(probSecondKingAfterFirstKing)


0.07692307692307693
0.004524886877828055
0.058823529411764705


BlackJack - probability natural 21 counting with permutations:

In [36]:
# given a bool tensor => we do OR and check for rank given in pos
def rankConditions(boolT, ranks, posHandRanks):
    if(len(ranks) == 0):
        return boolT
    nextRank = ranks.pop()
    nextCondition = posHandRanks == nextRank
    if(boolT == None):
        return rankConditions(nextCondition, ranks, posHandRanks)
    else:
        return rankConditions(boolT | nextCondition, ranks, posHandRanks)

def handsWithRanksAt(hands, pos, ranks):
    posHandRanks = hands[:, pos, 1]
    conditionApplied = rankConditions(None, ranks.copy(), posHandRanks)
    handsWithAnyOfRanksAtPos = torch.where(conditionApplied, torch.ones(1), torch.zeros(1)).nonzero()
    return torch.index_select(hands, 0, handsWithAnyOfRanksAtPos[:,0])

def naturalBlackJackProb(hands):
    ace = 14
    faceCards = [10,11,12,13]
    aceFirstHands = handsWithRanksAt(hands, 0, [ace])
    faceCardSecondWithAceFirstHand = handsWithRanksAt(aceFirstHands, 1, faceCards)
    faceCardFirstHands = handsWithRanksAt(hands, 0, faceCards)
    aceSecondWithFaceCardFirsts = handsWithRanksAt(faceCardFirstHands, 1, [ace])
    return faceCardSecondWithAceFirstHand.size()[0] / hands.size()[0] + aceSecondWithFaceCardFirsts.size()[0] / hands.size()[0]

print("Natural Black Jack probability is:")
print(naturalBlackJackProb(hands))


Natural Black Jack probability is:
0.048265460030165915


Natural Black Jack probability by sampling:

In [37]:
import pyro

# input tensor need to be of floars
deck_f = torch.div(deck, 1.)

def draw2():
    return pyro.ops.stats.resample(deck_f, 2)

sampled_hands = torch.stack([draw2() for _ in range(10000)])
print("Natural Black Jack probability is:")
print(naturalBlackJackProb(sampled_hands))

Natural Black Jack probability is:
0.048299999999999996
